<a href="https://colab.research.google.com/github/Mubarok982/Sentimen-Analisis/blob/main/Sentimen_Analisis_pengguna_apk_By_U.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Library**

In [10]:
!pip install google-play-scraper

#menghimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store
from google_play_scraper import app, reviews, Sort, reviews_all # Corrected package name

import pandas as pd
pd.set_option('mode.chained_assignment', None) #menonaktifkan peringatan chaining
import numpy as np
seed = 0
np.random.seed(seed) #Mengatur seed untuk reproduksibilitas
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import datetime as dt # Memanipulasi data waktu dan tanggal
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Assuming 'workcloud' was a typo and it should be 'wordcloud'
# You might need to install this library if it's not already available
# !pip install wordcloud
from wordcloud import WordCloud


import nltk
nltk.download('punkt') # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.5 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# **Scrapping Dataset**

In [17]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all

#Menggali semua ulasan aplikasi dengan ID "com.by.id" di google Play Store.
scrapeview = reviews_all(
    'com.byu.id',
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=1000  #Jumlah maksimum ulasan
)

# **Menyimpan ulasan dalam format csv**

In [18]:
import csv

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(['review', 'score'])
  for review in scrapeview:
    writer.writerow([review['content'], review['score']])

# **Loading Dataset**

In [20]:
app_reviews_df = pd.DataFrame(scrapeview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('app_reviews.csv', index=False)

#Membuat dataframe dari hasil scrapeview
app_reviews_df = pd.DataFrame(scrapeview)

#menghitung jumlah baris dan kolom dalam dataframe
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

#menampilkan jumlah ulasan dan kolom
print(f"Jumlah ulasan: {jumlah_ulasan}")
print(f"Jumlah kolom: {jumlah_kolom}")

Jumlah ulasan: 108000
Jumlah kolom: 11


In [21]:
# Menampilkan lima baris pertama dari DataFrame app_reviews_df
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,e5eac52c-4732-4f66-ade0-88eb212c4073,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"saya pengguna lama Byu, ini kenapa sayaa login...",1,4,1.61.0,2025-06-24 17:26:28,"Hi, Kak Apz. Maaf udah bikin ga nyaman. Boleh ...",2025-06-25 08:38:23,1.61.0
1,7b9f3040-48d4-488c-8f5d-a7b026703711,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sering eror, padahal jaringan bagus tapi apk n...",3,0,1.61.0,2025-06-24 18:01:00,"Hi, Kak Vigo. Maaf udah bikin ga nyaman. Boleh...",2025-06-25 08:46:50,1.61.0
2,46e4b7e2-4ee7-44f4-b5bc-c2d2a41a7dad,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"kurang bijak dalam melakukan tindakan, saya ba...",4,7,1.61.0,2025-06-24 23:00:40,"Hi, Kak Jul. Maaf banget udah bikin gak nyaman...",2025-06-25 10:18:51,1.61.0
3,04e5494c-7e14-49a9-85c3-059ba21ef3e1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"suda bagus... paketnya cukup murah²,sangat coc...",3,0,1.61.0,2025-06-24 21:52:08,"Dear Kak Efrem, maaf banget udah bikin interne...",2025-06-25 09:56:47,1.61.0
4,3749ca7f-1e96-4b49-89b5-f0c12b7f9579,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,dari merk lain pindah ke by u biar lebih oke d...,1,1,1.61.0,2025-06-24 20:15:43,"Hi Kak Rian Basuki, maaf udah bikin internetan...",2025-06-25 08:39:21,1.61.0


In [22]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108000 entries, 0 to 107999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              108000 non-null  object        
 1   userName              108000 non-null  object        
 2   userImage             108000 non-null  object        
 3   content               108000 non-null  object        
 4   score                 108000 non-null  int64         
 5   thumbsUpCount         108000 non-null  int64         
 6   reviewCreatedVersion  88501 non-null   object        
 7   at                    108000 non-null  datetime64[ns]
 8   replyContent          100228 non-null  object        
 9   repliedAt             100228 non-null  datetime64[ns]
 10  appVersion            88501 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 9.1+ MB


## **Menghapus baris kosong**

In [24]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81762 entries, 0 to 107999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              81762 non-null  object        
 1   userName              81762 non-null  object        
 2   userImage             81762 non-null  object        
 3   content               81762 non-null  object        
 4   score                 81762 non-null  int64         
 5   thumbsUpCount         81762 non-null  int64         
 6   reviewCreatedVersion  81762 non-null  object        
 7   at                    81762 non-null  datetime64[ns]
 8   replyContent          81762 non-null  object        
 9   repliedAt             81762 non-null  datetime64[ns]
 10  appVersion            81762 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 7.5+ MB


## **Menghapus baris duplikat dari dataframe**

In [25]:
clean_df = clean_df.drop_duplicates()

# Menghitung jumlah baris dan kolom dalam datframe clean_df setelah menghapus duplikat
jumlah_baris, jumlah_kolom = clean_df.shape

# Menampilkan jumlah baris dan kolom setelah menghapus duplikat
print(f"Jumlah baris setelah menghapus duplikat: {jumlah_baris}")
print(f"Jumlah kolom: {jumlah_kolom}")
clean_df.info()

Jumlah baris setelah menghapus duplikat: 78199
Jumlah kolom: 11
<class 'pandas.core.frame.DataFrame'>
Index: 78199 entries, 0 to 107999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              78199 non-null  object        
 1   userName              78199 non-null  object        
 2   userImage             78199 non-null  object        
 3   content               78199 non-null  object        
 4   score                 78199 non-null  int64         
 5   thumbsUpCount         78199 non-null  int64         
 6   reviewCreatedVersion  78199 non-null  object        
 7   at                    78199 non-null  datetime64[ns]
 8   replyContent          78199 non-null  object        
 9   repliedAt             78199 non-null  datetime64[ns]
 10  appVersion            78199 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 7.2+ MB


# **Prepocessing Text**

In [30]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Mengunduh daftar kata-kata berhenti (stopwords) dalam bahasa Inggris
nltk.download('stopwords')

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka

    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text


def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## menghapus kata **slang**

In [31]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

## **Penerapan Prepocessing text**

In [33]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)

# Download the missing resource
import nltk
nltk.download('punkt_tab')

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [34]:
clean_df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,e5eac52c-4732-4f66-ade0-88eb212c4073,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"saya pengguna lama Byu, ini kenapa sayaa login...",1,4,1.61.0,2025-06-24 17:26:28,"Hi, Kak Apz. Maaf udah bikin ga nyaman. Boleh ...",2025-06-25 08:38:23,1.61.0,saya pengguna lama Byu ini kenapa sayaa login ...,saya pengguna lama byu ini kenapa sayaa login ...,saya pengguna lama byu ini kenapa sayaa login ...,"[saya, pengguna, lama, byu, ini, kenapa, sayaa...","[pengguna, byu, sayaa, login, aplikasi, kesala...",pengguna byu sayaa login aplikasi kesalahan ja...
1,7b9f3040-48d4-488c-8f5d-a7b026703711,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sering eror, padahal jaringan bagus tapi apk n...",3,0,1.61.0,2025-06-24 18:01:00,"Hi, Kak Vigo. Maaf udah bikin ga nyaman. Boleh...",2025-06-25 08:46:50,1.61.0,Sering eror padahal jaringan bagus tapi apk ny...,sering eror padahal jaringan bagus tapi apk ny...,sering eror padahal jaringan bagus tapi apk ny...,"[sering, eror, padahal, jaringan, bagus, tapi,...","[eror, jaringan, bagus, apk, yg, susah, bug, b...",eror jaringan bagus apk yg susah bug buka apk ...
2,46e4b7e2-4ee7-44f4-b5bc-c2d2a41a7dad,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"kurang bijak dalam melakukan tindakan, saya ba...",4,7,1.61.0,2025-06-24 23:00:40,"Hi, Kak Jul. Maaf banget udah bikin gak nyaman...",2025-06-25 10:18:51,1.61.0,kurang bijak dalam melakukan tindakan saya bar...,kurang bijak dalam melakukan tindakan saya bar...,kurang bijak dalam melakukan tindakan saya bar...,"[kurang, bijak, dalam, melakukan, tindakan, sa...","[bijak, tindakan, semalam, isi, pulsa, trus, b...",bijak tindakan semalam isi pulsa trus bngt tau...
3,04e5494c-7e14-49a9-85c3-059ba21ef3e1,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"suda bagus... paketnya cukup murah²,sangat coc...",3,0,1.61.0,2025-06-24 21:52:08,"Dear Kak Efrem, maaf banget udah bikin interne...",2025-06-25 09:56:47,1.61.0,suda bagus paketnya cukup murah²sangat cocok u...,suda bagus paketnya cukup murah²sangat cocok u...,suda bagus paketnya cukup murah²sangat cocok u...,"[suda, bagus, paketnya, cukup, murah²sangat, c...","[suda, bagus, paketnya, murah²sangat, cocok, g...",suda bagus paketnya murah²sangat cocok gen z s...
4,3749ca7f-1e96-4b49-89b5-f0c12b7f9579,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,dari merk lain pindah ke by u biar lebih oke d...,1,1,1.61.0,2025-06-24 20:15:43,"Hi Kak Rian Basuki, maaf udah bikin internetan...",2025-06-25 08:39:21,1.61.0,dari merk lain pindah ke by u biar lebih oke d...,dari merk lain pindah ke by u biar lebih oke d...,dari merk lain pindah ke by u biar lebih oke d...,"[dari, merk, lain, pindah, ke, by, u, biar, le...","[merk, pindah, u, biar, oke, praktis, jaringan...",merk pindah u biar oke praktis jaringan gajela...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107995,5df6b2c1-c970-44c5-828f-c83b8c7d15d7,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus apk nya buruan download man teman,5,0,1.61.0,2025-06-13 12:16:53,"Hai Kak Bayu, makasih banyak buat rating dan u...",2025-06-13 12:18:29,1.61.0,bagus apk nya buruan download man teman,bagus apk nya buruan download man teman,bagus apk nya buruan download man teman,"[bagus, apk, nya, buruan, download, man, teman]","[bagus, apk, buruan, download, man, teman]",bagus apk buruan download man teman
107996,010963d7-be4c-4c34-88b1-d3deee4a30e8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,min tambain fitur top up game dong,5,0,1.61.0,2025-05-18 02:02:13,"Hi Kak, Makasih ya feedbacknya. Nindy dan tim ...",2025-05-18 02:47:04,1.61.0,min tambain fitur top up game dong,min tambain fitur top up game dong,min tambain fitur top up game d

# **Pelabelan**

In [36]:
import csv
import requests
from io import StringIO
from bs4 import BeautifulSoup

#Membaca data kamus kata-kata positif dari github
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatlan file csv darei github

if response.status_code == 200:
  #jika permintaan berhasil
  reader = csv.reader(StringIO(response.text), delimiter=',')
  # Membaca teks respon debagai file csv menggunakan pembaca csv dengan pemisah koma

  for row in reader:
    #mengulangi setiap baris dalam file csv
    lexicon_positive[row[0]] = int(row[1])
    # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positif
else:
  print("gagal fetching data ke lexicon positif")

# Membuat data kamus kata-kata negatif dari github
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [37]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

# **hasil**

In [38]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
negative    42382
positive    35817
Name: count, dtype: int64


# **Eksplorasi Label**

## **Data Splitting dan Ekstraksi Fitur dengan TF-IDF**

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# **MODELLING**

## **Naive Bayes**

In [41]:
from sklearn.naive_bayes import BernoulliNB

# Membuat objek model Naive Bayes (Bernoulli Naive Bayes)
naive_bayes = BernoulliNB()

# Melatih model Naive Bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

# Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

# Menampilkan akurasi
print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)

Naive Bayes - accuracy_train: 0.7891750187822696
Naive Bayes - accuracy_test: 0.7929667519181586


## **Random Forest**

In [42]:
from sklearn.ensemble import RandomForestClassifier

# Membuat objek model Random Forest
random_forest = RandomForestClassifier()

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.96903722885596
Random Forest - accuracy_test: 0.8531329923273657


## **Logistic Regression**

In [43]:
from sklearn.linear_model import LogisticRegression

# Membuat objek model Logistic Regression
logistic_regression = LogisticRegression()

# Melatih model Logistic Regression pada data pelatihan
logistic_regression.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_lr = logistic_regression.predict(X_train.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# Evaluasi akurasi model Logistic Regression pada data pelatihan
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)

# Evaluasi akurasi model Logistic Regression pada data uji
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

# Menampilkan akurasi
print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

Logistic Regression - accuracy_train: 0.8737511788871305
Logistic Regression - accuracy_test: 0.875


## **Decision tree**

In [44]:
from sklearn.tree import DecisionTreeClassifier

# Membuat objek model Decision Tree
decision_tree = DecisionTreeClassifier()

# Melatih model Decision Tree pada data pelatihan
decision_tree.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_dt = decision_tree.predict(X_train.toarray())
y_pred_test_dt = decision_tree.predict(X_test.toarray())

# Evaluasi akurasi model Decision Tree
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

# Menampilkan akurasi
print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)

Decision Tree - accuracy_train: 0.9690532137662047
Decision Tree - accuracy_test: 0.8065856777493606
